## LIMPIEZA DE DATOS ARCHIVOS CSV:


Vamos a comenzar con la limpieza de los datos archivo por archivo. Todos los datos están contenidos en archivos CSV, se trata de los siguientes:

- Denuncias: contiene datos relativos a las denuncias interpuestas por violencia de género por año, trimestre, provincia y origen.
- Llamadas al 016: contiene el total de llamadas recibidas por el 016 por año, trimestre, provincia y persona que realiza la llamada.
- Órdenes de protección: contiene el total de órdenes por provincia, año y trimestre.
- Menores: se trata de los menores asesinados por violencia de género. Contiene también año, trimestre y provincia, rango de edad del menor, si el agresor era el padre biológico o no y si este se suicidó.
- Víctimas: son las mujeres asesinadas por violencia de género. Contiene también año, trimestre, provincia, comunidad, edades de víctima y agresor, si eran pareja o había convivencia, si se produjo suicidio posterior y origen de ambos.
- Tipo de delito: recoge las distintas tipologías de delitos cometidas, dentro del marco de la violencia de género, por CCAA y año.
- Población total CCAA: extraemos los datos de población total de mujeres para poder calcular las tasas y realizar el análisis en relación al resto de datos.
- Población total provincia: contiene la misma información que el anterior pero para las provincias.


Todos los archivos se someten a un proceso de limpieza en el que unificamos los nombres de todas las provincias y comunidades para poder hacer combinaciones posteriormente, así como el formato de los años y el resto de datos numéricos. En algún caso, transformamos las categorías para unificar y facilitar el análisis.



In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../src')
from suport import unif_col
from suport import lower_tildes
from suport import clean_year
from suport import trimestre
from suport import eliminar_alfa  
from suport import ccaa
from suport import comunidades
from suport import eliminar_num
from suport import provincias
from suport import crear_ccaa
from unidecode import unidecode
pd.set_option('display.max_columns', None) 
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import json


### 1. LIMPIEZA ARCHIVO DENUNCIAS POR VIOLENCIA DE GÉNERO:


In [2]:
denu = pd.read_csv('../data/denuncias.csv')
denu.head()

,Año,Trimestre,Origen de la denuncia,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Gipuzkoa,Bizkaia,La Rioja,Ceuta,Melilla
0,Año 2009,Primero,Presentada directamente por victima,40.0,140.0,11.0,111.0,14.0,17.0,100.0,126.0,7.0,30.0,19.0,9.0,137.0,11.0,130.0,44.0,7.0,24.0,20.0,2.0,120.0,3.0,0.0,9.0,11.0,21.0,27.0,3.0,57.0,103.0,173.0,37.0,15.0,212.0,57.0,9.0,327.0,70.0,107.0,17.0,15.0,19.0,83.0,252.0,58.0,5.0,9.0,24.0,34.0,51.0,0.0,2.0
1,Año 2009,Primero,Presentada directamente por familiares,1.0,17.0,0.0,0.0,0.0,0.0,4.0,8.0,4.0,0.0,1.0,1.0,17.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,2.0,11.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Año 2009,Primero,Atestados policiales - con denuncia victima,419.0,549.0,307.0,748.0,437.0,313.0,837.0,731.0,63.0,9.0,409.0,427.0,650.0,828.0,624.0,158.0,71.0,148.0,138.0,68.0,22.0,47.0,38.0,283.0,57.0,176.0,176.0,85.0,209.0,201.0,1929.0,378.0,202.0,271.0,1159.0,189.0,898.0,104.0,89.0,298.0,76.0,172.0,453.0,4089.0,958.0,187.0,173.0,230.0,397.0,100.0,57.0,61.0
3,Año 2009,Primero,Atestados policiales - con denuncia familiar,7.0,8.0,6.0,4.0,0.0,7.0,28.0,3.0,0.0,0.0,8.0,3.0,15.0,8.0,5.0,18.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,1.0,39.0,1.0,0.0,11.0,21.0,8.0,2.0,0.0,0.0,9.0,1.0,2.0,1.0,25.0,27.0,9.0,1.0,2.0,19.0,2.0,0.0,0.0
4,Año 2009,Primero,Atestados policiales - por intervención direct...,82.0,30.0,13.0,15.0,25.0,30.0,210.0,38.0,8.0,0.0,72.0,20.0,107.0,91.0,43.0,28.0,17.0,33.0,47.0,3.0,3.0,11.0,1.0,77.0,0.0,20.0,25.0,0.0,25.0,21.0,535.0,96.0,8.0,115.0,267.0,51.0,780.0,7.0,10.0,32.0,8.0,3.0,0.0,831.0,175.0,36.0,5.0,12.0,71.0,34.0,7.0,9.0


In [3]:
denu.columns = unif_col(denu.columns)
denu['trimestre'] = lower_tildes(denu['trimestre'])
denu.isna().value_counts()

ano    trimestre  origen de la denuncia  almeria  cadiz  cordoba  granada  huelva  jaen   malaga  sevilla  huesca  teruel  zaragoza  asturias  illes balears  las palmas  santa cruz de tenerife  cantabria  avila  burgos  leon   palencia  salamanca  segovia  soria  valladolid  zamora  albacete  ciudad real  cuenca  guadalajara  toledo  barcelona  girona  lleida  tarragona  alicante/alacant  castellon/castello  valencia/valencia  badajoz  caceres  a coruna  lugo   ourense  pontevedra  madrid  murcia  navarra  araba/alava  gipuzkoa  bizkaia  la rioja  ceuta  melilla
False  False      False                  False    False  False    False    False   False  False   False    False   False   False     False     False          False       False                   False      False  False   False  False     False      False    False  False       False   False     False        False   False        False   False      False   False   False      False             False               False              

In [4]:
denu.rename(columns={'ano': 'año'}, inplace=True)
denu.año = clean_year(denu.año)

In [5]:
denu.rename(columns={'origen de la denuncia': 'origen_denuncia'}, inplace=True)
denu['origen_denuncia'] = lower_tildes(denu['origen_denuncia'])

In [6]:
denu['origen_denuncia'].unique()

array(['presentada directamente por victima',
       'presentada directamente por familiares',
       'atestados policiales - con denuncia victima',
       'atestados policiales - con denuncia familiar',
       'atestados policiales - por intervencion directa policial',
       'parte de lesiones', 'servicios asistencia-terceros  en general'],
      dtype=object)

In [7]:
origen = {'presentada directamente por victima': 'victima',
          'presentada directamente por familiares': 'familiar',
          'atestados policiales - con denuncia victima': 'policia_victima',
          'atestados policiales - con denuncia familiar': 'policia_familiar',
          'atestados policiales - por intervencion directa policial': 'intervencion_policial_directa',
          'parte de lesiones': 'parte_lesiones',
          'servicios asistencia-terceros  en general': 'asistencia_terceros'}

denu['origen_denuncia'] = denu['origen_denuncia'].map(origen)

In [8]:
col_int = denu.columns[3:]

denu[col_int] = denu[col_int].astype(int)

In [9]:
denu = pd.melt(denu, id_vars=['año', 'trimestre', 'origen_denuncia'], 
                var_name='provincia', value_name='total_denuncias')

In [10]:
denu

,año,trimestre,origen_denuncia,provincia,total_denuncias
0,2009,primero,victima,almeria,40
1,2009,primero,familiar,almeria,1
2,2009,primero,policia_victima,almeria,419
3,2009,primero,policia_familiar,almeria,7
4,2009,primero,intervencion_policial_directa,almeria,82
...,...,...,...,...,...
21107,2023,segundo,policia_victima,melilla,80
21108,2023,segundo,policia_familiar,melilla,1
21109,2023,segundo,intervencion_policial_directa,melilla,7
21110,2023,segundo,parte_lesiones,melilla,5


In [11]:
denu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21112 entries, 0 to 21111
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   año              21112 non-null  int32 
 1   trimestre        21112 non-null  object
 2   origen_denuncia  21112 non-null  object
 3   provincia        21112 non-null  object
 4   total_denuncias  21112 non-null  int64 
dtypes: int32(1), int64(1), object(3)
memory usage: 742.3+ KB


### 2. LIMPIEZA ARCHIVO LLAMADAS 016:

In [12]:
llam = pd.read_csv('../data/llamadas.csv')
llam.head()

,Año,Trimestre,Llamante,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Bizkaia,Gipuzkoa,La Rioja,Ceuta,Melilla,No consta
0,Año 2007,Tercero,Usuaria,3.0,2.0,7.0,6.0,4.0,NaN,8.0,16.0,1.0,NaN,4.0,5.0,6.0,20.0,6.0,8.0,3.0,NaN,2.0,2.0,1.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,27.0,3.0,1.0,4.0,8.0,2.0,4.0,3.0,4.0,5.0,NaN,2.0,2.0,99.0,6.0,4.0,1.0,2.0,1.0,NaN,1.0,1.0,16.0
1,Año 2007,Tercero,Familiares/Personas allegadas,NaN,NaN,3.0,1.0,2.0,NaN,5.0,9.0,NaN,NaN,NaN,2.0,1.0,3.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN,NaN,1.0,5.0,NaN,4.0,2.0,1.0,1.0,NaN,NaN,1.0,30.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,3.0
2,Año 2007,Tercero,Otros,NaN,NaN,NaN,1.0,2.0,2.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,4.0,2.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,20.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,3.0
3,Año 2007,Cuarto,Usuaria,150.0,187.0,155.0,250.0,82.0,131.0,357.0,608.0,37.0,40.0,152.0,248.0,309.0,365.0,298.0,177.0,60.0,65.0,128.0,44.0,80.0,48.0,25.0,159.0,36.0,50.0,93.0,34.0,35.0,180.0,1229.0,90.0,83.0,134.0,498.0,102.0,592.0,160.0,78.0,317.0,56.0,64.0,157.0,3368.0,386.0,123.0,62.0,195.0,76.0,49.0,22.0,18.0,123.0
4,Año 2007,Cuarto,Familiares/Personas allegadas,27.0,44.0,26.0,41.0,17.0,17.0,78.0,135.0,8.0,2.0,35.0,51.0,43.0,71.0,49.0,23.0,12.0,10.0,16.0,8.0,25.0,1.0,8.0,22.0,2.0,13.0,23.0,7.0,9.0,36.0,247.0,24.0,15.0,21.0,75.0,21.0,125.0,26.0,19.0,45.0,9.0,1.0,33.0,604.0,61.0,32.0,16.0,38.0,19.0,12.0,2.0,3.0,28.0


In [13]:
llam.columns = unif_col(llam.columns)
llam['trimestre'] = lower_tildes(llam['trimestre'])
llam.llamante = lower_tildes(llam.llamante)
llam.rename(columns={'ano': 'año'}, inplace=True)
llam.año = clean_year(llam.año)

In [14]:
llam.llamante.unique()

array(['usuaria', 'familiares/personas allegadas', 'otros', 'desconocido',
       'no consta'], dtype=object)

In [15]:
llam.columns

Index(['año', 'trimestre', 'llamante', 'almeria', 'cadiz', 'cordoba',
       'granada', 'huelva', 'jaen', 'malaga', 'sevilla', 'huesca', 'teruel',
       'zaragoza', 'asturias', 'illes balears', 'las palmas',
       'santa cruz de tenerife', 'cantabria', 'avila', 'burgos', 'leon',
       'palencia', 'salamanca', 'segovia', 'soria', 'valladolid', 'zamora',
       'albacete', 'ciudad real', 'cuenca', 'guadalajara', 'toledo',
       'barcelona', 'girona', 'lleida', 'tarragona', 'alicante/alacant',
       'castellon/castello', 'valencia/valencia', 'badajoz', 'caceres',
       'a coruna', 'lugo', 'ourense', 'pontevedra', 'madrid', 'murcia',
       'navarra', 'araba/alava', 'bizkaia', 'gipuzkoa', 'la rioja', 'ceuta',
       'melilla', 'no consta'],
      dtype='object')

In [16]:
quien_llama = {'usuaria': 'usuaria',
               'familiares/personas allegadas': 'familiar',
               'otros': 'desconocido',
               'desconocido': 'desconocido',
               'no Consta': 'desconocido'}

llam['llamante'] = llam['llamante'].map(quien_llama)
llam.llamante.value_counts()

llamante
desconocido    84
usuaria        65
familiar       65
Name: count, dtype: int64

In [17]:
llam.isna().value_counts()

año    trimestre  llamante  almeria  cadiz  cordoba  granada  huelva  jaen   malaga  sevilla  huesca  teruel  zaragoza  asturias  illes balears  las palmas  santa cruz de tenerife  cantabria  avila  burgos  leon   palencia  salamanca  segovia  soria  valladolid  zamora  albacete  ciudad real  cuenca  guadalajara  toledo  barcelona  girona  lleida  tarragona  alicante/alacant  castellon/castello  valencia/valencia  badajoz  caceres  a coruna  lugo   ourense  pontevedra  madrid  murcia  navarra  araba/alava  bizkaia  gipuzkoa  la rioja  ceuta  melilla  no consta
False  False      False     False    False  False    False    False   False  False   False    False   False   False     False     False          False       False                   False      False  False   False  False     False      False    False  False       False   False     False        False   False        False   False      False   False   False      False             False               False              False    False 

In [18]:
llam['llamante'] = llam['llamante'].fillna('desconocido')
llam.iloc[:, 3:] = llam.iloc[:, 3:].fillna(0)

In [19]:
col_int = llam.columns[3:]

llam[col_int] = llam[col_int].astype(int)

In [20]:
llam = pd.melt(llam, id_vars=['año', 'trimestre', 'llamante'], 
                var_name='provincia', value_name='total_llamadas')

In [21]:
llam.llamante.unique()

array(['usuaria', 'familiar', 'desconocido'], dtype=object)

In [22]:
llam.head()

,año,trimestre,llamante,provincia,total_llamadas
0,2007,tercero,usuaria,almeria,3
1,2007,tercero,familiar,almeria,0
2,2007,tercero,desconocido,almeria,0
3,2007,cuarto,usuaria,almeria,150
4,2007,cuarto,familiar,almeria,27


In [23]:
llam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12508 entries, 0 to 12507
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   año             12508 non-null  int32 
 1   trimestre       12508 non-null  object
 2   llamante        12508 non-null  object
 3   provincia       12508 non-null  object
 4   total_llamadas  12508 non-null  int64 
dtypes: int32(1), int64(1), object(3)
memory usage: 439.9+ KB


### 3. LIMPIEZA ARCHIVO ORDEN PROTECCIÓN

In [24]:
ord = pd.read_csv('../data/orden_proteccion.csv')
ord.head()

,Año,Trimestre,Provincia,Número de órdenes de protección
0,Año 2009,Primero,Almería,82.0
1,Año 2009,Primero,Cádiz,312.0
2,Año 2009,Primero,Córdoba,125.0
3,Año 2009,Primero,Granada,163.0
4,Año 2009,Primero,Huelva,138.0


In [25]:
ord.columns = unif_col(ord.columns)
ord.rename(columns={'ano': 'año'}, inplace=True)
ord.rename(columns={'numero de ordenes de proteccion': 'numero_ordenes_proteccion'}, inplace=True)
ord.año = clean_year(ord.año)

In [26]:
ord.trimestre = lower_tildes(ord.trimestre)
ord.provincia = lower_tildes(ord.provincia)
ord.isna().value_counts()

año    trimestre  provincia  numero_ordenes_proteccion
False  False      False      False                        3016
Name: count, dtype: int64

In [27]:
ord['numero_ordenes_proteccion'] = ord['numero_ordenes_proteccion'].astype(int)

In [28]:
ord.head()

,año,trimestre,provincia,numero_ordenes_proteccion
0,2009,primero,almeria,82
1,2009,primero,cadiz,312
2,2009,primero,cordoba,125
3,2009,primero,granada,163
4,2009,primero,huelva,138


In [29]:
ord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3016 entries, 0 to 3015
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   año                        3016 non-null   int32 
 1   trimestre                  3016 non-null   object
 2   provincia                  3016 non-null   object
 3   numero_ordenes_proteccion  3016 non-null   int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 82.6+ KB


### 4. LIMPIEZA ARCHIVO MENORES VÍCTIMAS MORTALES DE VIOLENCIA DE GÉNERO:

In [39]:
men = pd.read_csv('../data/menores.csv')
men.head()

,Año,Trimestre,Tramo de edad de la víctima,Padre biológico,Suicidio,Almería,Cádiz,Jaén,Málaga,Teruel,Asturias,Illes Balears,Santa Cruz de Tenerife,Cantabria,Valladolid,Ciudad Real,Barcelona,Alicante/Alacant,Castellón/Castelló,Valencia/València,A Coruña,Pontevedra,Madrid,Murcia
0,Año 2013,Primero,3-4 años,Sí,No hubo tentativa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Año 2013,Segundo,<1 año,Sí,Suicidio consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Año 2013,Segundo,5-6 años,Sí,Suicidio consumado,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Año 2013,Segundo,11-12 años,Sí,Suicidio consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Año 2013,Tercero,5-6 años,Sí,No hubo tentativa,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
men.columns = unif_col(men.columns)
men.rename(columns={'padre biologico': 'padre_biologico'}, inplace=True)
men.rename(columns={'ano': 'año'}, inplace=True)
men.rename(columns={'tramo de edad de la victima': 'edad'}, inplace=True)

men['padre_biologico'] = lower_tildes(men['padre_biologico'])
men.trimestre = lower_tildes(men.trimestre)
men.suicidio = lower_tildes(men.suicidio)
men.edad = eliminar_alfa(men.edad)
men.año = clean_year(men.año)
men.fillna(0, inplace = True)

In [41]:
men.suicidio.unique()

array(['no hubo tentativa', 'suicidio consumado',
       'tentativa no consumada'], dtype=object)

In [42]:
suicidio = {'no hubo tentativa': 'no',
               'suicidio consumado': 'si',
               'tentativa no consumada': 'tentativa'}

men.suicidio = men.suicidio.map(suicidio)

In [43]:
men.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   año                     48 non-null     int32  
 1   trimestre               48 non-null     object 
 2   edad                    48 non-null     object 
 3   padre_biologico         48 non-null     object 
 4   suicidio                48 non-null     object 
 5   almeria                 48 non-null     float64
 6   cadiz                   48 non-null     float64
 7   jaen                    48 non-null     float64
 8   malaga                  48 non-null     float64
 9   teruel                  48 non-null     float64
 10  asturias                48 non-null     float64
 11  illes balears           48 non-null     float64
 12  santa cruz de tenerife  48 non-null     float64
 13  cantabria               48 non-null     float64
 14  valladolid              48 non-null     floa

In [44]:
col_int = men.columns[5:]

men[col_int] = men[col_int].astype(int)

In [45]:
men = pd.melt(men, id_vars=['año', 'trimestre', 'edad', 'padre_biologico', 'suicidio'], 
                var_name='provincia', value_name='total_menores_vict_mortales')

In [46]:
men['total_menores_vict_mortales'].sum()

50

In [47]:
men.tail()

,año,trimestre,edad,padre_biologico,suicidio,provincia,total_menores_vict_mortales
907,2021,cuarto,3-4,si,si,murcia,0
908,2022,segundo,11-12,si,no,murcia,0
909,2022,cuarto,5-6,si,tentativa,murcia,0
910,2023,primero,7-8,no,tentativa,murcia,0
911,2023,cuarto,5-6,si,tentativa,murcia,0


### 5. LIMPIEZA ARCHIVO VÍCTIMAS MORTALES POR VIOLENCIA DE GÉNERO

In [48]:
vic = pd.read_csv('../data/victimas_mortales.csv')
vic.head()

,Año,Mes,Pareja - Expareja,Convivencia,Denuncia agresor,Suicidio,Tramo de edad Agresor,Tramo de edad Víctima,País de nacimiento Agresor,País de nacimiento Víctima,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,Huesca,Teruel,Zaragoza,Asturias,Illes Balears,Las Palmas,Santa Cruz de Tenerife,Cantabria,Ávila,Burgos,León,Palencia,Salamanca,Segovia,Soria,Valladolid,Zamora,Albacete,Ciudad Real,Cuenca,Guadalajara,Toledo,Barcelona,Girona,Lleida,Tarragona,Alicante/Alacant,Castellón/Castelló,Valencia/València,Badajoz,Cáceres,A Coruña,Lugo,Ourense,Pontevedra,Madrid,Murcia,Navarra,Araba/Álava,Bizkaia,Gipuzkoa,La Rioja,Ceuta,Melilla
0,Año 2003,Enero,Expareja,No,No consta denuncia,No hubo tentativa,No consta,31-40 años,España,España,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,31-40 años,31-40 años,Otro país,España,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,51-60 años,41-50 años,España,España,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,No consta,21-30 años,No consta,No consta,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Año 2003,Enero,Pareja,Sí,No consta denuncia,No hubo tentativa,No consta,31-40 años,España,España,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# Formateamos sin minusculas ni tildes y modificamos nulos:
vic.fillna(0, inplace = True)
vic = vic.apply(lower_tildes)
vic.columns = unif_col(vic.columns)
vic.head()

,ano,mes,pareja - expareja,convivencia,denuncia agresor,suicidio,tramo de edad agresor,tramo de edad victima,pais de nacimiento agresor,pais de nacimiento victima,almeria,cadiz,cordoba,granada,huelva,jaen,malaga,sevilla,huesca,teruel,zaragoza,asturias,illes balears,las palmas,santa cruz de tenerife,cantabria,avila,burgos,leon,palencia,salamanca,segovia,soria,valladolid,zamora,albacete,ciudad real,cuenca,guadalajara,toledo,barcelona,girona,lleida,tarragona,alicante/alacant,castellon/castello,valencia/valencia,badajoz,caceres,a coruna,lugo,ourense,pontevedra,madrid,murcia,navarra,araba/alava,bizkaia,gipuzkoa,la rioja,ceuta,melilla
0,ano 2003,enero,expareja,no,no consta denuncia,no hubo tentativa,no consta,31-40 anos,espana,espana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,31-40 anos,31-40 anos,otro pais,espana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,51-60 anos,41-50 anos,espana,espana,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,no consta,21-30 anos,no consta,no consta,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ano 2003,enero,pareja,si,no consta denuncia,no hubo tentativa,no consta,31-40 anos,espana,espana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# Renombramos las siguientes columnas:
vic.rename(columns={'ano': 'año'}, inplace=True)
vic.rename(columns={'tramo de edad agresor': 'edad_agresor'}, inplace=True)
vic.rename(columns={'tramo de edad victima': 'edad_victima'}, inplace=True)
vic.rename(columns={'pareja - expareja': 'pareja'}, inplace=True)
vic.rename(columns={'ano': 'año'}, inplace=True)
vic.rename(columns={'pais de nacimiento agresor': 'nacimiento_agresor'}, inplace=True)
vic.rename(columns={'pais de nacimiento victima': 'nacimiento_victima'}, inplace=True)
vic.rename(columns={'denuncia agresor': 'denuncia'}, inplace=True)

In [51]:
# Transformamos valores en las filas:
vic['edad_agresor'] = eliminar_alfa(vic['edad_agresor'])
vic['edad_victima'] = eliminar_alfa(vic['edad_victima'])
vic.año = clean_year(vic.año)
vic.suicidio = vic.suicidio.map(suicidio)


In [52]:
vic.pareja.unique()

array(['expareja', 'pareja', 'pareja en fase de separacion'], dtype=object)

In [53]:
pareja = {'expareja': 'no',
          'pareja': 'si',
          'pareja en fase de separacion': 'en_separacion'}

vic.pareja = vic.pareja.map(pareja)

In [54]:
vic.denuncia.unique()

array(['no consta denuncia', 'no habia denuncia', 'habia denuncia'],
      dtype=object)

In [55]:
denuncia = {'no consta denuncia': 'desconocido',
          'no habia denuncia': 'no',
          'habia denuncia': 'si'}

vic.denuncia = vic.denuncia.map(denuncia)

In [56]:
vic['edad_victima'].unique()

array(['31-40', '41-50', '21-30', '71-84', '', '61-70', '51-60', '18-20',
       '>=85', '<16', '16-17'], dtype=object)

In [57]:
vic['edad_victima'] = vic['edad_victima'].replace('', 'desconocido')
vic['edad_agresor'] = vic['edad_agresor'].replace('', 'desconocido')

In [58]:
vic['nacimiento_victima'].unique()

array(['espana', 'no consta', 'otro pais'], dtype=object)

In [59]:
nacim = {'espana': 'españa',
         'no consta': 'desconocido',
         'otro pais': 'otro'}


vic['nacimiento_victima'] = vic['nacimiento_victima'].map(nacim)
vic['nacimiento_agresor'] = vic['nacimiento_agresor'].map(nacim)

In [60]:
# Añadimos columna de trimestre:
vic['trimestre'] = vic['mes'].apply(trimestre)

# Vamos a reordenar las columnas:
columnas = vic.columns.tolist()

# Extraemos la última columna de trimestre:
ultima_columna = columnas[-1]
ultima_columna
# La movemos a la tercera posición para que tenga consistencia:
columnas.insert(2, columnas.pop(-1))

# Reordenamos:
vic = vic[columnas]

In [61]:
vic = pd.melt(vic, id_vars=['año', 'mes', 'trimestre', 'pareja', 'convivencia',
                              'denuncia', 'suicidio', 'edad_agresor', 'edad_victima',
                              'nacimiento_agresor', 'nacimiento_victima'], 
                var_name='provincia', value_name='total_victimas_mortales')

In [62]:
vic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63856 entries, 0 to 63855
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   año                      63856 non-null  int32 
 1   mes                      63856 non-null  object
 2   trimestre                63856 non-null  object
 3   pareja                   63856 non-null  object
 4   convivencia              63856 non-null  object
 5   denuncia                 63856 non-null  object
 6   suicidio                 63856 non-null  object
 7   edad_agresor             63856 non-null  object
 8   edad_victima             63856 non-null  object
 9   nacimiento_agresor       63856 non-null  object
 10  nacimiento_victima       63856 non-null  object
 11  provincia                63856 non-null  object
 12  total_victimas_mortales  63856 non-null  object
dtypes: int32(1), object(12)
memory usage: 6.1+ MB


In [63]:
# Cambiamos el tipo de dato de las columnas que tienen los totales

vic['total_victimas_mortales'] = vic['total_victimas_mortales'].apply(pd.to_numeric, errors='coerce')
vic['total_victimas_mortales'] = vic['total_victimas_mortales'].astype(int)

In [184]:
vic.head()

,año,mes,trimestre,pareja,convivencia,denuncia,suicidio,edad_agresor,edad_victima,nacimiento_agresor,nacimiento_victima,provincia,comunidad,total_victimas_mortales
0,2003,enero,primero,no,no,desconocido,no,desconocido,31-40,españa,españa,almeria,andalucia,0
1,2003,enero,primero,si,si,desconocido,no,31-40,31-40,otro,españa,almeria,andalucia,0
2,2003,enero,primero,si,si,desconocido,no,51-60,41-50,españa,españa,almeria,andalucia,1
3,2003,enero,primero,si,si,desconocido,no,desconocido,21-30,desconocido,desconocido,almeria,andalucia,0
4,2003,enero,primero,si,si,desconocido,no,desconocido,31-40,españa,españa,almeria,andalucia,0


In [185]:
vic['edad_agresor'].unique()

array(['desconocido', '31-40', '51-60', '71-84', '61-70', '41-50',
       '21-30', '>=85', '18-20', '16-17'], dtype=object)

In [186]:
vic['edad_victima'].unique()

array(['31-40', '41-50', '21-30', '71-84', 'desconocido', '61-70',
       '51-60', '18-20', '>=85', '<16', '16-17'], dtype=object)

### 6. LIMPIEZA ARCHIVO TIPOS DE DELITOS COMETIDOS

In [74]:
tipos = pd.read_csv('../data/tipos.csv', encoding= 'utf-8', sep=';')
tipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2736 entries, 0 to 2735
Data columns (total 4 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Tipo de infracción                2736 non-null   object 
 1   Comunidades y ciudades autónomas  2736 non-null   object 
 2   Periodo                           2736 non-null   int64  
 3   Total                             2584 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 85.6+ KB


In [75]:
# Formateamos sin minusculas ni tildes y borramos nulos:
tipos = tipos.apply(lower_tildes)
tipos.columns = unif_col(tipos.columns)

In [76]:
tipos.isna().value_counts()

tipo de infraccion  comunidades y ciudades autonomas  periodo  total
False               False                             False    False    2736
Name: count, dtype: int64

In [77]:
tipos.dropna(how='any', inplace=True) 

In [78]:
tipos.rename(columns={'tipo de infraccion': 'tipo'}, inplace=True)
tipos.rename(columns={'comunidades y ciudades autonomas': 'comunidad'}, inplace=True)
tipos.rename(columns={'periodo': 'año'}, inplace=True)

In [79]:
tipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2736 entries, 0 to 2735
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tipo       2736 non-null   object
 1   comunidad  2736 non-null   object
 2   año        2736 non-null   object
 3   total      2736 non-null   object
dtypes: object(4)
memory usage: 85.6+ KB


In [80]:
tipos.tipo.unique()

array(['delitos', 'homicidio y sus formas', 'lesiones',
       'detenciones ilegales y secuestro', 'amenazas', 'coacciones',
       'torturas e integridad moral', 'agresiones sexuales',
       'abusos sexuales',
       'abusos y agresiones sexuales a menores de 16 anos',
       'allanamiento de morada', 'injurias', 'danos',
       'quebrantamiento de condena', 'otros delitos sin especificar',
       'faltas', 'faltas contra las personas',
       'otras faltas sin especificar'], dtype=object)

In [81]:
tipologia = {'delitos': 'delito', 
             'homicidio y sus formas': 'homicidio', 
             'lesiones': 'lesiones',
             'detenciones ilegales y secuestro': 'detencion_secuestro', 
             'amenazas': 'amenazas', 
             'coacciones': 'coacciones',
             'torturas e integridad moral': 'tortura_integridad_moral', 
             'agresiones sexuales': 'agresion_sexual',
             'abusos sexuales': 'abuso_sexual',
             'abusos y agresiones sexuales a menores de 16 anos': 'abuso_menores',
             'allanamiento de morada': 'allanamiento', 
             'injurias': 'injurias', 
             'danos': 'daños',
             'quebrantamiento de condena': 'quebrantamiento_condena', 
             'otros delitos sin especificar': 'otros',
             'faltas': 'faltas', 
             'faltas contra las personas': 'faltas',
             'otras faltas sin especificar': 'faltas'}

tipos.tipo = tipos.tipo.map(tipologia)

In [82]:
tipos.tipo.unique()

array(['delito', 'homicidio', 'lesiones', 'detencion_secuestro',
       'amenazas', 'coacciones', 'tortura_integridad_moral',
       'agresion_sexual', 'abuso_sexual', 'abuso_menores', 'allanamiento',
       'injurias', 'daños', 'quebrantamiento_condena', 'otros', 'faltas'],
      dtype=object)

In [83]:
tipos['total'].value_counts()

total
0.0      481
1.0      222
nan      152
2.0      146
3.0       88
        ... 
1.502      1
1.523      1
1.725      1
1.425      1
312.0      1
Name: count, Length: 617, dtype: int64

In [84]:
tipos = tipos.sort_values(by = 'año').reset_index(drop = True)

In [85]:
tipos.head()

,tipo,comunidad,año,total
0,abuso_sexual,melilla,2015,0.0
1,agresion_sexual,"asturias, principado de",2015,3.0
2,agresion_sexual,"balears, illes",2015,1.0
3,agresion_sexual,canarias,2015,1.0
4,agresion_sexual,cantabria,2015,0.0


In [86]:

tipos['total'] = tipos['total'].str.replace('.', '').replace('nan', float('nan'), regex=False)
tipos['total'] = pd.to_numeric(tipos['total'], errors='coerce')

tipos['total'].fillna(0, inplace=True)
tipos['total'] = tipos['total'].astype(int)


In [87]:
tipos.comunidad.unique()

array(['melilla', 'asturias, principado de', 'balears, illes', 'canarias',
       'cantabria', 'castilla y leon', 'castilla - la mancha', 'cataluna',
       'comunitat valenciana', 'extremadura', 'galicia',
       'madrid, comunidad de', 'murcia, region de',
       'navarra, comunidad foral de', 'pais vasco', 'rioja, la', 'aragon',
       'ceuta', 'andalucia'], dtype=object)

### 7. LIMPIEZA ARCHIVO POBLACION TOTAL POR COMUNIDAD AUTÓNOMA


In [168]:
total_ccaa = pd.read_csv('../data/total_ccaa.csv', encoding= 'Latin1', sep = '\t')
total_ccaa = total_ccaa.drop(columns=['Nacional'])
total_ccaa = total_ccaa.dropna()
total_ccaa.head()

,Comunidades y Ciudades Autónomas,Sexo,Periodo,Total
30,01 Andalucía,Hombres,2021,4.173.339
31,01 Andalucía,Hombres,2020,4.170.605
32,01 Andalucía,Hombres,2019,4.147.167
33,01 Andalucía,Hombres,2018,4.133.898
34,01 Andalucía,Hombres,2017,4.133.835


In [169]:
total_ccaa = total_ccaa.apply(lower_tildes)
total_ccaa.columns = unif_col(total_ccaa.columns)
total_ccaa.rename(columns={'comunidades y ciudades autonomas': 'comunidad', 
                     'periodo': 'año'}, inplace=True)

In [170]:
total_ccaa = total_ccaa.sort_values(by = 'año').reset_index(drop = True)

In [171]:
total_ccaa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   comunidad  570 non-null    object
 1   sexo       570 non-null    object
 2   año        570 non-null    object
 3   total      570 non-null    object
dtypes: object(4)
memory usage: 17.9+ KB


In [172]:
total_ccaa['total'] = total_ccaa['total'].str.replace('.', '')
total_ccaa['total'] = pd.to_numeric(total_ccaa['total'], errors='coerce')

In [173]:
total_ccaa.head()

,comunidad,sexo,año,total
0,10 comunitat valenciana,hombres,2007,2432162
1,09 cataluna,mujeres,2007,3632332
2,09 cataluna,hombres,2007,3578176
3,08 castilla - la mancha,mujeres,2007,981636
4,08 castilla - la mancha,hombres,2007,995668


### 8. LIMPIEZA ARCHIVO CSV POBLACION TOTAL POR PROVINCIAS

In [174]:
total_prov = pd.read_csv('../data/total_provincias.csv', encoding= 'Latin1', sep = '\t')
total_prov.head()

,Provincias,Sexo,Periodo,Total
0,02 Albacete,Hombres,2021,193.205
1,02 Albacete,Hombres,2020,194.081
2,02 Albacete,Hombres,2019,193.911
3,02 Albacete,Hombres,2018,194.158
4,02 Albacete,Hombres,2017,194.743


In [175]:
total_prov = total_prov.apply(lower_tildes)
total_prov.columns = unif_col(total_prov.columns)
total_prov.rename(columns={'provincias': 'provincia', 
                     'periodo': 'año'}, inplace=True)

In [176]:
total_prov = total_prov.sort_values(by = 'año').reset_index(drop = True)

In [177]:
total_prov['total'] = total_prov['total'].str.replace('.', '')
total_prov['total'] = pd.to_numeric(total_prov['total'], errors='coerce')

In [178]:
total_prov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   provincia  1560 non-null   object
 1   sexo       1560 non-null   object
 2   año        1560 non-null   object
 3   total      1560 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 48.9+ KB


In [179]:
total_prov.head()

,provincia,sexo,año,total
0,23 jaen,mujeres,2007,334199
1,14 cordoba,mujeres,2007,403009
2,"15 coruna, a",hombres,2007,544352
3,"15 coruna, a",mujeres,2007,588440
4,16 cuenca,hombres,2007,106633


#### NORMALIZAMOS TODAS LAS COLUMNAS DE PROVINCIA Y COMUNIDAD AUTÓNOMA DE TODOS LOS CSV:

In [180]:
# Normalizamos la columna de provincia de todos los dataframes:

provin = [denu, llam, ord, men, vic, total_prov]

provin_normalizado = provincias(provin)
denu, llam, ord, men, vic, total_prov = provin_normalizado

In [181]:
# Creamos la columna de comunidad donde solo hubiese la de provincia:
provins = [denu, llam, ord, men, vic]
comun = crear_ccaa(provins)
denu, llam, ord, men, vic = comun

In [182]:
# Normalizamos la columna de comunidad del resto de dataframes que la contienen:

comuni = [tipos, total_ccaa]
normalizar = comunidades(comuni)
tipos, total_ccaa = normalizar

#### CARGAMOS TODOS LOS DATAFRAMES LIMPIOS EN ARCHIVOS CSV:

In [183]:
denu.to_csv('../data_clean/portal_estadistico_vio_gen/denuncias.csv', index=False)
llam.to_csv('../data_clean/portal_estadistico_vio_gen/llamadas016.csv', index=False)
ord.to_csv('../data_clean/portal_estadistico_vio_gen/ordenes_prot.csv', index=False)
men.to_csv('../data_clean/portal_estadistico_vio_gen/menores.csv', index=False)
vic.to_csv('../data_clean/portal_estadistico_vio_gen/victimas_mortales.csv', index=False)
tipos.to_csv('../data_clean/ine/tipos_violencias.csv', index=False)
total_ccaa.to_csv('../data_clean/ine/total_ccaa.csv', index=False)
total_prov.to_csv('../data_clean/ine/total_prov.csv', index=False)

RESCATAMOS ARCHIVO DE LA EXTRACCIÓN WEB SCRAPING PARA UTILIZAR MÁS TARDE EN STREAMLIT


El archivo recoge una serie de leyes en materia de violencia de género por Comunidad Autónoma.

In [3]:

with open('/Users/noeliarosonmartin/Ironhack/final_project/data_clean/scrapeo/leyes.json', 'r') as archivo_json:
    leyes = json.load(archivo_json)

# Comprobamos que contiene lo que nos interesa
print(leyes)




{'País Vasco': ['Ley 4/2005, de 18 de febrero, para la Igualdad de Mujeres y Hombres, modificada por la Ley 1/2022, de 3 de marzo, de segunda modificación de la Ley para la Igualdad de Mujeres y Hombres.'], 'Cataluña': ['Ley 5/2008, de 24 de abril, del derecho de las mujeres a erradicar la violencia machista, modificada por la Ley 17/2020, de 22 de diciembre, de modificación de la Ley 5/2008, del derecho de las mujeres a erradicar la violencia machista.', 'Ley 17/2015, de 21 de julio, de igualdad efectiva de mujeres y hombres'], 'Galicia': ['Ley 11/2007, de 27 de julio, gallega para la prevención y el tratamiento integral de la violencia de género, modificada por la Ley 15/2021, de 3 de diciembre, por la que se modifica la Ley 11/2007, de 27 de julio, gallega para la prevención y el tratamiento integral de la violencia de género.', 'Decreto Legislativo 2/2015, de 12 de febrero, por el que se aprueba el texto refundido de las disposiciones legales de la Comunidad Autónoma de Galicia en 